In [ ]:
!pip install requests

!pip install beautifulsoup4

!pip install numpy

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import numpy as np
from pprint import pprint
import json
import html.parser

In [ ]:
#парсинг MIT

In [ ]:
all_page = ['https://news.mit.edu/topic/artificial-intelligence2?page=0', ]
for i in range(1, 75):
  all_page.append(f'https://news.mit.edu/topic/artificial-intelligence2?page={i}')

In [ ]:
pur_url_lst = []

for i in all_page:
  html_text_page = requests.get(i).text
  soup = BeautifulSoup(html_text_page, 'html.parser')
  tag_url_artcl = soup.find_all('a', class_='term-page--news-article--item--title--link')
  for i in tag_url_artcl:
    pur_url_lst.append(f"https://news.mit.edu{i['href']}")

In [ ]:
art_data_dict = {'headers': [], 'time': [], 'text': []}

for art_url in pur_url_lst:

  url_art_rquest = requests.get(art_url).text
  soup_art = BeautifulSoup(url_art_rquest)
  art_data_dict['headers'].append(soup_art.find('div', id="block-mit-page-title").find('span').text)
  art_data_dict['time'].append(soup_art.find('time').text)
  art_data_dict['text'].append(soup_art.find('div', class_='news-article--content--body').text.replace('\n', ' '))

In [ ]:
with open('art_data.json', 'w', encoding='utf=8') as file:
  json.dump(art_data_dict, file, indent=4, ensure_ascii=False)

In [ ]:
df = pd.read_json('art_data.json')
df = pd.DataFrame(art_data_dict)
df.to_csv('m_art_data.csv', sep=',', encoding='utf-8')

In [ ]:
print(df)

In [ ]:
df = pd.read_csv('m_art_data.csv')
df.to_csv('m_art_data.txt', sep='\t', index=False)

df = df['headers']
df.to_csv('m_headers.txt', sep='\t', index=False)

df = pd.read_csv('m_art_data.csv')
df.to_csv('m_art_data.txt', sep='\t', index=False)

df = df['text']
df.to_csv('m_text.txt', sep='\t', index=False)

In [ ]:
# парсинг The economist

In [ ]:
all_page2 = ['https://www.economist.com/search?q=AI&page=1', ]
for i in range(1, 11):
  all_page2.append(f'https://www.economist.com/search?q=AI&page={i}')

In [ ]:
e_pur_url_set = set()

for i in all_page2:
  html_text_page = requests.get(i).text
  soup = BeautifulSoup(html_text_page, 'html.parser')
  e_tag_url_artcl = soup.find_all('a', class_='_search-result')

  for i in e_tag_url_artcl:
    e_pur_url_set.add(i['href'])

In [ ]:
e_art_data_dict = {'headers': [], 'time': [], 'text': []}

for e_art_url in e_pur_url_set:

  try:
    e_url_art_rquest = requests.get(e_art_url).text
    soup_art = BeautifulSoup(e_url_art_rquest)
    headers = soup_art.find('main', id="content").find('div', class_='css-1r2sn2n e1mdktgm0').find('h1', class_="css-sous8d e1phhnog0").text
    e_art_data_dict['headers'].append(headers)

    time = soup_art.find('time').text
    e_art_data_dict['time'].append(time)

    rex = re.search(r'css-tnm0wy ekpjo0f0|css-cbns87 ekpjo0f2', e_url_art_rquest)
    text = soup_art.find('div', class_=rex.group(0)).text.replace('\n', ' ').replace('\xa0', ' ')
    e_art_data_dict['text'].append(text)
  except AttributeError:
    pass

In [ ]:
with open('e_art_data.json', 'w', encoding='utf=8') as file:
  json.dump(e_art_data_dict, file, indent=4, ensure_ascii=False)

In [ ]:
df = pd.read_json('e_art_data.json')

In [ ]:
df

In [ ]:
df = pd.DataFrame(e_art_data_dict)
e_df = pd.DataFrame(e_art_data_dict)
e_df.to_csv('e_art_data.csv', sep=',', encoding='utf-8')

e_df = pd.read_csv('e_art_data.csv')
e_df.to_csv('e_art_data.txt', sep='\t', index=False)

print(e_df)

In [ ]:
e_df = df['headers']
e_df.to_csv('e_headers.txt', sep='\t', index=False)

e_df = pd.read_csv('e_art_data.csv')
e_df.to_csv('e_art_data.txt', sep='\t', index=False)

e_df = df['text']
e_df.to_csv('e_text.txt', sep='\t', index=False)

In [ ]:
# парсинг BBC

In [ ]:
page_lst = [f'https://www.bbc.co.uk/search?q=AI&d=SEARCH_PS&seqId=a377c420-d300-11ee-807f-ab7c7d65eaf8&page={i}' for i in range(1, 30)]

In [ ]:
b_art_url_lst = []

for pg in page_lst:
    html_txt_pg = requests.get(pg).text
    soup = BeautifulSoup(html_txt_pg, 'html.parser')
    srch_url = soup.find_all('a', class_='ssrcss-its5xf-PromoLink exn3ah91')

    for i in srch_url:
        b_art_url_lst.append(i['href'])

In [ ]:
pprint(len(b_art_url_lst))

In [ ]:
b_art_data_dict = {'headers': [], 'time': [], 'text': []}

reg_comp = re.compile(r'\d\d\d\d-\d\d-\d\d')

for art in b_art_url_lst:

    try:
        url_art_rquest = requests.get(art).text
        soup_art = BeautifulSoup(url_art_rquest)
        time = soup_art.find('span', class_='ssrcss-1if1g9v-MetadataText e4wm5bw1').find('time').get('datetime')
        b_art_data_dict['time'].append(re.search(reg_comp, time).group(0))
        b_art_data_dict['headers'].append(soup_art.find('h1', id="main-heading").text)
        txt = [p.text for p in soup_art.find_all('p', class_='ssrcss-1q0x1qg-Paragraph e1jhz7w10')]
        b_art_data_dict['text'].append(str(txt))
    except AttributeError:
        pass

In [ ]:
with open('b_art_data.json', 'w', encoding='utf=8') as file:
    json.dump(b_art_data_dict, file, indent=4, ensure_ascii=False)

In [ ]:
df = pd.read_json('b_art_data.json')

In [ ]:
df = pd.DataFrame(b_art_data_dict)
b_df = pd.DataFrame(b_art_data_dict)
b_df.to_csv('b_art_data.csv', sep=',', encoding='utf-8')

b_df = pd.read_csv('b_art_data.csv')
b_df.to_csv('b_art_data.txt', sep='\t', index=False)

print(b_df)

In [ ]:
b_df = df['headers']
b_df.to_csv('b_headers.txt', sep='\t', index=False)

b_df = pd.read_csv('b_art_data.csv')
b_df.to_csv('b_art_data.txt', sep='\t', index=False)

b_df = df['text']
b_df.to_csv('b_text.txt', sep='\t', index=False)

In [ ]:
# объединение заголовков и текстов The Economist & BBC

In [ ]:
open("Media_headers.txt","w").write(open("e_headers.txt","r").read() + open("b_headers.txt","r").read())

In [ ]:
open("Media_text.txt","w").write(open("e_text.txt","r").read() + open("b_text.txt","r").read())

In [ ]:
# парсинг fox news

In [ ]:
with open('fox_news.txt', 'r') as file:
    article_html = file.read()

In [ ]:
fxn_soup = BeautifulSoup(article_html, 'html.parser')

In [ ]:
href_artic = fxn_soup.find_all('h2', class_="title")

In [ ]:
fxn_artcl_url_lst = [i.find('a').get('href') for i in href_artic]

In [ ]:
fxn_art_data_dict = {'headers': [], 'time': [], 'text': []}

for url in fxn_artcl_url_lst:
    try:
        url_art_rquest = requests.get(url).text
        soup_art = BeautifulSoup(url_art_rquest)
        fxn_art_data_dict['time'].append(soup_art.find('span', class_="article-date").find('time').text.strip())
        fxn_art_data_dict['headers'].append(soup_art.find('h1', class_="headline speakable").text)
        txt = [i.text for i in soup_art.find('div', class_='article-body').find_all('p')]
        fxn_art_data_dict['text'].append(str(txt))
    except AttributeError:
        pass

In [ ]:
df = pd.DataFrame(fxn_art_data_dict)
df.to_csv('fxn_art_data.csv', sep=',', encoding='utf-8')

In [ ]:
df = pd.read_csv('fxn_art_data.csv')
df.to_csv('fxn_art_data.txt', sep='\t', index=False)

In [ ]:
df = df['headers']
df.to_csv('fxn_headers.txt', sep='\t', index=False)

df = pd.read_csv('fxn_art_data.csv')
df.to_csv('fxn_art_data.txt', sep='\t', index=False)

df = df['text']
df.to_csv('fxn_text.txt', sep='\t', index=False)